# SIT 720 Assignment 2
Bryon Baker

Student number: 85031775

# Install required libraries

In [ ]:
!pip3 install pandas
!pip3 install numpy
!pip3 install matplotlib
!pip3 install yellowbrick

In [ ]:
import random
import pandas as pd # dataframe manipulation
import numpy as np # linear algebra

# data visualization
import matplotlib.pyplot as plt
%matplotlib inline
from yellowbrick.cluster import KElbowVisualizer # cluster visualizer

# sklearn kmeans
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import contingency_matrix

# pyclustering kmeans


In [ ]:
# Seed the random nunmber generator to ensure K-Means randomisation works properly.
random.seed()

print(f"Seeding random number generator. Random number is: {random.random()}")

#np.set_printoptions(threshold=np.inf)

# Part 2: SCADI Dataset

## Question 5
* Download the attached heartdisease.csv file. 
* Read the file and separate the class and feature matrix.

In [ ]:
#unpruned_df = unpruned_df.replace('?', np.nan)
#print(unpruned_df)
#full_df = unpruned_df[~unpruned_df.eq("?").any(1)]
# The ? in the missing values sets the column type to object. Change the column to numeric.
#numericFeatures = pd.Series(["ca", "thal"], copy = False)
#full_df[numericFeatures] = full_df[numericFeatures].apply(pd.to_numeric)

In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

url = 'https://raw.githubusercontent.com/bryonbaker/datasets/main/SIT720/Ass2/heartdisease.csv'
unpruned_df = pd.read_csv(url)

print("Shape of data upon read: {}".format(unpruned_df.shape))
# Drop the rows with missing values
unpruned_df = unpruned_df.apply (pd.to_numeric, errors='coerce')# Anything non-numeric is NaN
print(unpruned_df)
unpruned_df = unpruned_df.dropna()
full_df = unpruned_df.reset_index(drop=True)

# Display meta data about the dataframe.
print("Shape of data post row pruning: {}\n".format(full_df.shape))
print("Dataframe structure:\n{}".format(full_df.dtypes))

# Separate the features form the labels
X = full_df.drop(['num'], axis=1)
y = full_df['num']
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

# Display information about the contents to aid quality inspection.
print("\n"+"*"*75)
print("Dataframe Content Summary\n"+"*"*75)
print("\nFeatures:\n"+"="*10+"\n{}".format(X))
print("\nClasses:\n"+"="*10+"\n{}".format(y))

## Question 6
* Determine the number of clusters from the dataset. Is this the same as the actual number of classes in the dataset?

In [ ]:
# Scan through the labels and count the number of unique values.
num_labels = y.nunique()
print("INFO: The breakdown of labels in the dataset are:\n{}".format(y.value_counts()))

# Find the optimal number of clusters for the dataset
model = KMeans()
elbowChart = KElbowVisualizer(model, k=(1, 11), timings=False)

elbowChart.fit(X) # Fit the data to the visualiser
elbowChart.show() # Plot the graph and the elbow
plt.show()
optimalClusters = elbowChart.elbow_value_

print("Cluster/Label relationship:")
print(f"The number of unique labels in the dataset is: {num_labels}")
print(f"Optimal number of clusters is: {optimalClusters}")

# Question 7:

Perform K-Means clustering on the complete dataset and report the purity score.

In [ ]:
# Takes a list that may contain duplicates and returns a list with unique ites.
def uniqueList(list1):
    # insert the list to the set
    list_set = set(list1)
    # convert the set to the list
    unique_list = (list(list_set))
    unique_list.sort()
    return unique_list

# Searches a Series and finds the item that appears with the greatest frequency and returns the count.
def findMaxLabel( data ):
    unique, counts = np.unique(data, return_counts=True)
    # print(np.asarray((unique, counts)).T)
    idx = np.argmax(counts, axis=0)
    # print(f"Index of max: {idx}")

    return unique[idx], counts[idx]

# Given a dataframe of columns "cluster" and "label", it calculates the cluster purity
def calculatePurity(data):
    print("INFO: Calculating clustering purity.")
    #print("INFO: Input data:\n{}".format(data))
          
    assert data.shape[1] > 0        # Check we won't have a divide by zero error later

    # Get a list of clusters that has no duplicates so we can use it as a key.
    clusterList = uniqueList(data['cluster'])
    # print(f"Cluster list: {clusterList}")

    # Go through each cluster one by one.
    runningSum = 0
    for c in clusterList:
        # print(f"INFO: Counting instances in cluster {c}")
        m1,m2 = findMaxLabel(data[data["cluster"] == c]["label"])
        print("INFO: Maximum frequency in cluster {} is label {}. It has a count of {}.".format(c,m1,m2))
        runningSum = runningSum + m2

    purity = runningSum / data.shape[0]

    return purity

In [ ]:
# instatiate KMeans class and set the number of clusters to the optimal number returned form the elbow chart
model = KMeans(n_clusters=optimalClusters, random_state=10)

#X.reset_index(drop=True)
# call fit method with data 
km_result = model.fit(X)

# coordinates of cluster center
centroids = model.cluster_centers_ 

In [ ]:
# The K-Means clustering algorithm preserves the order of the input data in the resultant labels. Using the clusters defined during training, 
# Create a nx2 table of the identified clusters and the class labels in the input dataset. Use this to calculate the cluster purity.
ls = pd.Series(y).astype(int)
ys = pd.Series(km_result.labels_).astype(int)
d = pd.concat([ys,ls], axis=1)
d.columns = ['cluster', 'label']

# print("Allocation of labels to clusters is:\n{}".format(d))
purity = calculatePurity(d)
print(f"\nNumber of clusters usefd for KMeans: {optimalClusters}")
print("Total number of data points in clusters is {}".format(y.shape[0]))
print("Purity of clustering is: {:.4f}%".format(purity))

## Discussion:
The data set is very imbalanced, there are
* 164 elements with label 0
* 55 with label 1
* 36 with label 2
* 35 with label 3
* 13 with label 4

As a result, it is not surprising that label 0 is overshadowing the purity of the clustering

# Question 8. 
There are several distance metrics for K-Means such as euclidean, squared euclidian, Manhattan, Chebyshev, Minowski.
  * Your job is to compare the purity score of k-means clustering for different distance metrics
  * Select the best distance metric and explain why this distance metric is best for the given dataset.

In [ ]:
# Reference: 
#    The following code snippet is based on the work of Arush Chillar, 2021
#    Blog url: https://www.kaggle.com/code/arushchillar/kmeans-clustering-using-different-distance-metrics/notebook

from pyclustering.cluster.kmeans import kmeans
from pyclustering.utils.metric import distance_metric
from pyclustering.cluster.center_initializer import random_center_initializer
from pyclustering.cluster.encoder import type_encoding
from pyclustering.cluster.encoder import cluster_encoder

# define dictionary for distance measures
distance_measures = {'euclidean': 0, 'squared euclidean': 1, 'manhattan': 2, 'chebyshev': 3, 
                    'canberra': 5, 'chi-square': 6}

# PyClustering uses an array and not a dataframe so need to convert the type.
numpy_X = X.to_numpy()

print(f"\nINFO: Number of clusters used for KMeans: {optimalClusters}")
print("INFO: Total number of data points in clusters is {}".format(numpy_X.shape[0]))

results_df = pd.DataFrame(columns=['purity', 'metric'])

for measure, value in distance_measures.items():
    print("*"*50)
    print(f"INFO: Calculating purity for {measure} distance\n")
    
    initial_centers = random_center_initializer(numpy_X, optimalClusters, random_state=10).initialize()
    # instance created for respective distance metric
    instanceKm = kmeans(numpy_X, initial_centers=initial_centers, metric=distance_metric(value))
    # perform cluster analysis
    instanceKm.process()
    # cluster analysis results - clusters and centers
    pyClusters = instanceKm.get_clusters()
    # pyCenters = instanceKm.get_centers()
    # enumerate encoding type to index labeling to get labels
    pyEncoding = instanceKm.get_cluster_encoding()
    pyEncoder = cluster_encoder(pyEncoding, pyClusters, numpy_X)
    pyLabels = pyEncoder.set_encoding(0).get_clusters()
    
    # calculatePurity requires a 2D list so combin the two lists/Series into a single 2D daraframe
    d = pd.concat([y,pd.Series(pyLabels)], axis=1)
    d.columns = ['cluster', 'label']
    purity = calculatePurity(d)
    results_df = results_df.append({'purity':purity,'metric':measure}, ignore_index = True)
    
    print("INFO: The purity score for {} distance is {:.4f}%".format(measure, purity))
    print("*"*50+"\n")

In [ ]:
print("*"*50)
print("SUMMARY")
print("*"*50)
print(results_df)

max_purity_df = results_df[results_df.purity == results_df['purity'].max()]
max_purity_df = max_purity_df.reset_index()  # make sure indexes pair with number of rows
for index, row in max_purity_df.iterrows():
    print("The best distance metric(s) is: {} with {:.4}% purity.".format(row['metric'], row['purity']))

# QUESTION: 9
Use selection criteria (ANOVA, Chi-squared) to select the best three features and use them for K-Means clustering.
Based on the purity score, which features are you going to recommend and why?